## Model Isinga - symulacja Monte Carlo
autor: Bartosz Kreft (418184)

Model Isinga, znany również jako model magnetyczny Isinga, to prosty model matematyczny stosowany w fizyce statystycznej do opisania układów o spinie, takich jak magnetyki. Model ten został opracowany przez niemieckiego fizyka Ernsta Isinga w 1925 roku i od tego czasu znalazł szerokie zastosowanie w różnych dziedzinach nauki.

W modelu Isinga zakłada się, że spiny oddziałują jedynie ze swoimi najbliższymi sąsiadami. Wpływ tych oddziaływań jest opisywany przez parametr zwany siłą oddziaływania lub stałą sprzężenia. 

Rozmatrujemy spiny rozłożone na sieci kwadratowej o torusowych własnościach brzegowych. Spiny mogą przyjmować jedno z dwóch możliwych stanów: 
* "góra" (+1, $\uparrow$)
* "dół" (-1, $\downarrow$).

Spinami można interpretować na przykład atomowe magnesy, w których kierunek magnetyzacji może być skierowany w górę lub w dół. W zależności od kontekstu, spins mogą również reprezentować inne obiekty, takie jak cząsteczki w ciele stałym. Pełny hamniltonian układu ma postać:

$$\begin{equation*}
 \mathcal{H} = -J\sum_{\langle i,j \rangle}S_iS_j-H\sum_{i=1}^NS_i \;,
\end{equation*}$$
gdzie $S_i, S_j$ to wartości sąsiednich spinów w sieci, $N$ to wymiar sieci, parametr $H$ jest związany gdy w modelu występuje zewnętrzna. Natomiast parametr $J$ odpowiada za siłę oddziaływań pomiędzy sąsiednimi spinami:
* $J>0$ - ferromagnetyczne (ustawia spiny w jednym kierunku, przeciwnym do zewnętrznego pola),
* $J<0$ - antyferromagnetyczne,
* $J=0$ - para spinów nie oddziałuje ze sobą.

Podstawowym celem modelu Isinga jest opisanie termodynamicznych właściwości układu spinów, takich jak temperatura krytyczna, magnetyzacja czy energia swobodna. Istnieją różne metody matematyczne i numeryczne, takie jak metoda Monte Carlo, które pozwalają na analizę i symulację modelu Isinga w celu zrozumienia zachowania układu w różnych warunkach.

Model Isinga ma zastosowanie w wielu dziedzinach nauki, w tym w fizyce ciała stałego, fizyce statystycznej, teorii pola, naukach przyrodniczych i informatyce. Jego prostota i jednocześnie zdolność do opisywania złożonych zachowań czynią go popularnym narzędziem badawczym w różnych dziedzinach nauki.

### Założenia symulacji
W przedstawionej symulacji zakładamy, że $H=0$, czyli układ nieoddziaływuje z zewnętrznym polem magnetycznym oraz spiny przy krawędziach oddziaływują ze spinami z drugiej krawędzi, tak aby interakcje pomiędzy spianmi były zgodne z geometrią torusa.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import scienceplots
import numba
from numba import njit
from scipy.ndimage import convolve, generate_binary_structure
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter

plt.style.use(['science','notebook', 'grid'])

In [3]:
class Ising():
    ''' Symulacja modelu Isinga '''    
    
    def __init__(self, N=10, temp=0.4, J=1, NIter=1000):
        self.N = N
        self.temp = temp
        self.J = J
        self.NIter = NIter
        self.init_config = np.zeros((N, N))
        
    def MetropolisMove(self, config, N, beta):
        ''' Krok metody Monte Carlo przy pomocy 
        algorytmu Metropolis '''
        for i in range(N):
            for j in range(N):            
                # wybór lsowego spinu z sieci
                a = np.random.randint(0,N)
                b = np.random.randint(0,N)
                spin_i =  config[a, b] # początkiowy spin
       
                # Energia
                nb = config[(a+1)%N,b] + config[a,(b+1)%N] + config[(a-1)%N,b] + config[a,(b-1)%N]
                dE = 2*nb*spin_i
                
                # if (dE < 0) or (np.rand() < np.exp(-dE*beta)):	
                #     spin_i *= -1 # obracanie spinu

                if (dE < 0):	
                    spin_i *= -1 # obracanie spinu
                elif np.random.random() < np.exp(-dE*beta):
                    spin_i *= -1 # obracanie spinu

                config[a, b] = spin_i
        return config
    
    def GetAverageMagnetization(self, config):
        return np.sum(config)/len(config)**2

    def GetEnergy(self, config):
        '''Energia układu w danym stanie, spiny jedynie oddziałyywują w pierwszej strefie kordynacyjnej'''
        N = len(config)
        energy = 0
        lattice_energy = np.zeros(config.shape)
        for i in range(len(config)):
            for j in range(len(config)):
                S = config[i,j]
                nb = config[(i+1)%N, j] + config[i,(j+1)%N] + config[(i-1)%N, j] + config[i,(j-1)%N]
                energy += -self.J*nb*S
                lattice_energy[i,j] = -self.J*nb*S
        return energy/4., lattice_energy 

    def IsConfigStable(self, config):
        return np.abs(self.GetEnergy(config)[0]) == np.size(config)


    def Simulate(self):   
        ''' Faktyczna symulacja'''
        config = 2*np.random.randint(2, size=(self.N, self.N))-1
        fig, ax = plt.subplots(figsize=(5, 5), dpi=180)
        self.init_config = config
        
        ims = []
        for i in range(self.NIter):
            self.MetropolisMove(config, self.N, 1.0/self.temp)
            
            ax.set_xticks([])
            ax.set_yticks([])
            im = ax.imshow(config, interpolation='nearest', cmap=cm.gray, animated=True)
            ttl = plt.text(0.5, 1.01, "SYMULACJA MODELU ISINGA 2D \n Krok: "+str(i)+"  N="+str(self.N)+"  T="+str(self.temp)+"K", 
                        horizontalalignment='center', 
                        verticalalignment='bottom', 
                        transform=ax.transAxes,
                        fontsize='xx-large')

            # if self.IsConfigStable(config):
            #     break
            
            ims.append([im, ttl])

        animation.ArtistAnimation(fig, ims, 
                                  interval=120, 
                                  blit=True,
                                  repeat_delay=1000).save("Sims\Ising"+str(self.N)+".gif")
        plt.close()

    def GetNameOfSimulation(self):
        return "Ising"+str(self.N)+".gif"
#Funkcja pod wykresy E(T), M(T),specific heat, susceptibility

In [4]:
rm = Ising(N=10, NIter=int(10))
rm.Simulate()

MovieWriter ffmpeg unavailable; using Pillow instead.


In [5]:
lattice = rm.init_config
lattice # jakiś błąd związny z alokowanie pamięci ?

array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
       [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])

In [6]:
from IPython.display import Image, clear_output
clear_output(wait=True)
Image(url='./Sims/'+rm.GetNameOfSimulation())  

### Wielkości fizyczne związane z Modelem Isinga